In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import dlib
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
# 创建面部识别模型
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [3]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [4]:
# 定义三重损失
def triplet_loss(y_true, y_pred, alpha = 0.2):
   
    # anchor : 形状为(None, 128) 的原图像的编码，
    # posptive : 形状为(None, 128)的同一个人的图像的编码
    # negative : 形状为(None, 128)的不是同一个人的图像的编码
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # 计算两张同一个人之间的编码距离
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)))
    # 计算两张不是同一个人之间的编码距离
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)))
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # 计算总损失
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))
    
    return loss

In [5]:
# 加载训练模型
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [6]:
size = 96
friends_images_path = "./friends_images"
re_friends_images_path = "./re_friends_images"
#使用dlib自带的frontal_face_detector作为我们的特征提取器
detector = dlib.get_frontal_face_detector()
#处理图片
def re_image():
    for filename in os.listdir(friends_images_path):
        print (filename)
        im_path = friends_images_path + '/' + filename
        # 从文件读取图片
        img = cv2.imread(im_path)
        # 转为灰度图片
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 使用detector进行人脸检测 dets为返回的结果
        dets = detector(gray_img, 1)

        #使用enumerate 函数遍历序列中的元素以及它们的下标
        #下标i即为人脸序号
        #left：人脸左边距离图片左边界的距离 ；right：人脸右边距离图片左边界的距离
        #top：人脸上边距离图片上边界的距离 ；bottom：人脸下边距离图片上边界的距离
        for i, d in enumerate(dets):
            x1 = d.top() if d.top() > 0 else 0
            y1 = d.bottom() if d.bottom() > 0 else 0
            x2 = d.left() if d.left() > 0 else 0
            y2 = d.right() if d.right() > 0 else 0
            # img[y:y+h,x:x+w]
            face = img[x1:y1,x2:y2]
            # 调整图片的尺寸
            face = cv2.resize(face, (size,size))
            #cv2.imshow('image',face)
            # 保存图片
            cv2.imwrite(re_friends_images_path + '/' + filename, face)
        print ("OK!")

re_image()

Ariela Moreno_1.jpg
OK!
Ariela Moreno_2.jpg
OK!
Bibesh Rai_1.jpg
OK!
Fredericks King_1.jpg
OK!
Md Himayu_1.jpg
OK!
Md Himayu_2.jpg
OK!
Patrong Doekoe_1.jpg
OK!
Patrong Doekoe_2.jpg
OK!
Sanjay Romario Parsons_1.jpg
OK!
Shavaughn Henry_1.jpg
OK!
Shavaughn Henry_2.jpg
OK!
Winston Mendez_1.jpg
OK!


In [7]:
database = {}
def load_database():
    for filename in os.listdir(re_friends_images_path):
        name = filename.split('.')[0]
        path = re_friends_images_path + '/'+ filename
        database[name] = img_to_encoding(path, FRmodel)

load_database()
print (database)

{'Ariela Moreno_2': array([[  1.46676794e-01,   5.82923815e-02,  -4.49319258e-02,
         -6.48921803e-02,   1.86526805e-01,   1.46262541e-01,
          1.44120112e-01,   6.17259853e-02,  -1.85092971e-01,
          1.36700328e-02,   2.11254228e-04,   3.81934643e-02,
          1.70717403e-01,   1.76611189e-02,   6.42346889e-02,
         -2.15976182e-02,   1.10107079e-01,  -1.33307057e-03,
         -1.37341172e-01,   2.02699095e-01,   8.43131468e-02,
          1.59768425e-02,   1.01272780e-02,   1.36930738e-02,
         -6.97176978e-02,  -6.86561093e-02,  -1.85872614e-01,
         -5.14026769e-02,   8.61377642e-02,   8.59770402e-02,
         -8.41461718e-02,   8.61850902e-02,  -9.77224559e-02,
          9.24462751e-02,  -3.12187932e-02,   6.86100544e-03,
          1.74576193e-02,   1.32299751e-01,  -4.44326326e-02,
         -6.74196184e-02,   6.07565604e-02,   5.40559888e-02,
          2.74759401e-02,  -2.42770221e-02,  -1.83303937e-01,
          5.41489981e-02,   8.03842098e-02,   5.75

In [8]:
# 应用识别
def who_is_it(image_path, database, model):
    
    # 计算照片的编码
    encoding = img_to_encoding(image_path, model)
    # 定义一个最小的距离
    min_dist = 100
    
    # 循环database
    for (name, db_enc) in database.items():
        
        # 计算L2距离
        dist = np.linalg.norm(encoding - db_enc)

        # 如果距离小于min_dist, 更新min_dist和identity
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("此人为 ：" + str(identity) + ", 距离为：" + str(min_dist))
        
    return min_dist, identity

In [9]:
size = 96
predict_friends_images_path = "./predict_friends_images"
re_predict_friends_images_path = "./re_predict_friends_images"
#使用dlib自带的frontal_face_detector作为我们的特征提取器
detector = dlib.get_frontal_face_detector()
#处理图片
def re_image():
    for filename in os.listdir(predict_friends_images_path):
        print (filename)
        im_path = predict_friends_images_path + '/' + filename
        # 从文件读取图片
        img = cv2.imread(im_path)
        # 转为灰度图片
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 使用detector进行人脸检测 dets为返回的结果
        dets = detector(gray_img, 1)

        #使用enumerate 函数遍历序列中的元素以及它们的下标
        #下标i即为人脸序号
        #left：人脸左边距离图片左边界的距离 ；right：人脸右边距离图片左边界的距离
        #top：人脸上边距离图片上边界的距离 ；bottom：人脸下边距离图片上边界的距离
        for i, d in enumerate(dets):
            x1 = d.top() if d.top() > 0 else 0
            y1 = d.bottom() if d.bottom() > 0 else 0
            x2 = d.left() if d.left() > 0 else 0
            y2 = d.right() if d.right() > 0 else 0
            # img[y:y+h,x:x+w]
            face = img[x1:y1,x2:y2]
            # 调整图片的尺寸
            face = cv2.resize(face, (size,size))
            #cv2.imshow('image',face)
            # 保存图片
            cv2.imwrite(re_predict_friends_images_path + '/' + filename, face)
        print ("OK!")

re_image()

Ariela Moreno.jpg
OK!
Md Himayu.jpg
OK!
Patrong Doekoe.jpg
OK!


In [10]:
for filename in os.listdir(re_predict_friends_images_path):
    path = re_predict_friends_images_path + '/' + filename
    print ("正在预测图片" + filename)
    who_is_it(path, database, FRmodel)

正在预测图片Ariela Moreno.jpg
此人为 ：Patrong Doekoe_2, 距离为：0.698382
正在预测图片Md Himayu.jpg
此人为 ：Bibesh Rai_1, 距离为：0.608774
